In [8]:
import pandas as pd
import json
from pprint import pprint

from wikibaseintegrator.wbi_config import config as wbi_config
from wikibaseintegrator import wbi_login, WikibaseIntegrator, datatypes, wbi_helpers

wbi_config['MEDIAWIKI_API_URL'] = 'https://ppsdb.wikibase.cloud/w/api.php'
wbi_config['SPARQL_ENDPOINT_URL'] = 'https://ppsdb.wikibase.cloud/query/sparql'
wbi_config['WIKIBASE_URL'] = 'https://ppsdb.wikibase.cloud'

with open("secrets/bot_password.json", "r") as fh:
    pw = json.load(fh)
login = wbi_login.Login(user = pw['user'], password=pw['password'])

sparql_prefixes = """
PREFIX pp: <https://ppsdb.wikibase.cloud/entity/>
PREFIX ppt: <https://ppsdb.wikibase.cloud/prop/direct/>
PREFIX pps: <https://ppsdb.wikibase.cloud/prop/>
PREFIX ppss: <https://ppsdb.wikibase.cloud/prop/statement/>
PREFIX ppsq: <https://ppsdb.wikibase.cloud/prop/qualifier/>
PREFIX ppsr: <https://ppsdb.wikibase.cloud/prop/reference/>
"""

In [2]:
query = """
SELECT DISTINCT ?qid ?P29 WHERE {
  {
    SELECT DISTINCT ?item ?qid ?itemLabel ?maybeGenus WHERE {
      ?item ppt:P19 ?symbiont.
      FILTER(NOT EXISTS { ?item ppt:P29 ?parent. })
      BIND(ENCODE_FOR_URI(REPLACE(STR(?item), ".*Q", "Q")) AS ?qid)
      SERVICE wikibase:label {
        bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".
        ?item rdfs:label ?itemLabel.
      }
      BIND(STRBEFORE(?itemLabel, " ") AS ?maybeGenus)
    }
  }
  {
    SELECT DISTINCT ?item2 ?item2Label WHERE {
      ?item2 ppt:P32 pp:Q601;
        rdfs:label ?item2Label.
      FILTER((LANG(?item2Label)) = "en")
    }
  }
  FILTER(?item2Label = ?maybeGenus)
  BIND(ENCODE_FOR_URI(REPLACE(STR(?item), ".*Q", "Q")) AS ?qid)
  BIND(ENCODE_FOR_URI(REPLACE(STR(?item2), ".*Q", "Q")) AS ?P29)
}
"""

In [3]:
tolink = wbi_helpers.execute_sparql_query(query=query, prefix=sparql_prefixes)

In [6]:
wbi = WikibaseIntegrator(login=login)

In [7]:
for i in tolink['results']['bindings']:
    q = wbi.item.get(i['qid']['value'])
    q.claims.add(datatypes.Item(i['P29']['value'], prop_nr="P29"))
    q.write(summary="Link host taxon to genus by name")